In [ ]:

# ======================================================================
# AIRBNB PRICE PREDICTION PROJECT 
# ======================================================================
# This script loads Airbnb listing data, cleans and preprocesses it,
# engineers useful features, and trains machine learning models to 
# predict log-transformed listing prices. Two models are compared:
# 1) Linear Regression (baseline)
# 2) Random Forest Regressor (with hyperparameter tuning)
# ======================================================================

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# ==============================================================================
# Step 1: Load and Inspect Data
# ==============================================================================
print("Step 1: Loading the dataset...")
try:
    df = pd.read_excel('Airbnb.xlsx')                            # Loads out datafile
    print(" Data loaded successfully!  :) ")
    print("\nInitial Data Overview:")
    df.info()                                                    # Show the dataset's overview
    print("\nFirst 5 rows:")
    print(df.head())                                             # shows top 5 rows
    print("-" * 50)
except FileNotFoundError:
    raise FileNotFoundError(" 'Airbnb.xlsx' not found. Place the file in the current directory. :( ")

# ==============================================================================
# Step 2: Data Cleaning
# ==============================================================================
print("Step 2: Cleaning and preprocessing data...")

# Feature: Number of amenities
df['num_amenities'] = df['amenities'].fillna('').apply(           # To count number of amenities
    lambda x: len(x.strip('{}').split(',')) if x else 0
)

# Drop irrelevant/high-missing columns
drop_cols = [                                                      # Drops all the columns from dataset which have 
    'review_scores_rating', 'host_response_rate', 'first_review',  # irrelevant data
    'last_review', 'thumbnail_url', 'neighbourhood',
    'name', 'description', 'amenities', 'zipcode'
]
df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Impute missing values for numerical columns with median
for col in ['bathrooms', 'bedrooms', 'beds']:
    df[col] = df[col].fillna(df[col].median())

# Fill boolean-like columns
for col in ['host_has_profile_pic', 'host_identity_verified']:
    df[col] = df[col].fillna('f')

# Fill missing host_since with most frequent value
df['host_since'] = df['host_since'].fillna(df['host_since'].mode()[0])

# Drop rows with missing target
df.dropna(subset=['log_price'], inplace=True)

# ==============================================================================
# Step 3: Feature Transformation & Encoding
# ==============================================================================
print("Step 3: Transforming features...")

# Convert boolean-like columns to numeric
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0}).fillna(0).astype(int)
df['cleaning_fee'] = df['cleaning_fee'].fillna(0).astype(int)
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0}).fillna(0).astype(int)
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0}).fillna(0).astype(int)

# Host tenure feature
df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce').fillna(pd.to_datetime('today'))
df['host_tenure_days'] = (pd.to_datetime('today') - df['host_since']).dt.days
df.drop(columns='host_since', inplace=True)

# One-hot encoding for categorical variables, to avoid any data with text values, got this IDEA from AI
categorical_cols = ['property_type', 'room_type', 'cancellation_policy', 'city', 'bed_type']
categorical_cols = [col for col in categorical_cols if col in df.columns]
df_final = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(" Data preprocessing completed :) ")
print(df_final.info())
print("-" * 50)

# ==============================================================================
# Step 4: Model Training and Evaluation
# ==============================================================================
print("Step 4: Training models...")

# Define target and features
y = df_final['log_price']
X = df_final.drop(columns=['log_price'])

# Train-test split (80% training set and rest 20% testing set)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42 )


# ----------------- Linear Regression (Baseline) -----------------
print("Training Linear Regression...")
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)                                       # fitting data into model
y_pred_lin = lin_reg.predict(X_test)

mae_lin = mean_absolute_error(y_test, y_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
r2_lin = r2_score(y_test, y_pred_lin)

print(f"Linear Regression → MAE: {mae_lin:.2f}, RMSE: {rmse_lin:.2f}, R²: {r2_lin:.2f}")


# ----------------- Random Forest (Default) -----------------

print("Training Random Forest (default parameters)...")             # I have taken Default Random Forest Parameters here
rf_default = RandomForestRegressor(random_state=42)
rf_default.fit(X_train, y_train)
y_pred_rf = rf_default.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest → MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}, R²: {r2_rf:.2f}")

# ==============================================================================
# Step 5: Hyperparameter Tuning for Random Forest
# ==============================================================================
print("Step 5: Hyperparameter tuning for Random Forest...")

# Define parameter grid                  
param_grid = {                           
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# GridSearchCV setup
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    scoring='r2'
)

grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_

# Predictions with best model
y_pred_best_rf = best_rf.predict(X_test)
mae_best_rf = mean_absolute_error(y_test, y_pred_best_rf)
rmse_best_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_best_rf = r2_score(y_test, y_pred_best_rf)

print(f"Best RF Params: {grid_search.best_params_}")
print(f"Random Forest Tuned → MAE: {mae_best_rf:.2f}, R²: {r2_best_rf:.2f}")
print(f"Random Forest Tuned → RMSE: {rmse_best_rf:.2f}, R²: {r2_best_rf:.2f}")


# ==============================================================================
# Step 6: Visualization - Actual vs Predicted
# ==============================================================================
plt.figure(figsize=(12, 5))

# Linear Regression
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_lin, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Linear Regression: Actual vs Predicted')
plt.xlabel('Actual log_price')
plt.ylabel('Predicted log_price')

# Random Forest Tuned
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_best_rf, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Random Forest (Tuned): Actual vs Predicted')
plt.xlabel('Actual log_price')
plt.ylabel('Predicted log_price')

plt.tight_layout()
plt.show()




In [ ]:
#SUMMARY of Findings

# Two regression models were developed to predict Airbnb log-transformed prices:

# -> Linear Regression
# -> Random Forest Regressor (with hyperparameter tuning)

# Key metrics show Random Forest performs better:

# Linear Regression: MAE = 0.35, RMSE = 0.47, R² = 0.58
# Random Forest (Default): MAE = 0.28, RMSE = 0.39, R² = 0.70
# Random Forest (Tuned): MAE = 0.29, RMSE = 0.39, R² = 0.69

# CONCLUSION: Random Forest captures non-linear relationships better and explains ~70% of price variation.


# KEY INSIGHTS

# MODEL PERFORMANCE :

# -> Random Forest outperforms Linear Regression in MAE and R², meaning it predicts prices more accurately and generalizes 
#   better.
# -> The improvement comes from capturing complex interactions between features (e.g., city × property type).

# POTENTIAL PRICE INFLUENCERS 

# -> Number of amenities and property type have strong influence on pricing.
# ->Location (city) is a major driver — high-demand cities command higher prices.
# -> Host tenure days (experience) and instant booking availability also positively impact price.

# PREDICTION CAPABILITIES

# -> The tuned Random Forest model can predict prices for new listings with ~0.39 RMSE, translating to good practical 
# accuracy in actual currency terms.




# BUSINESS RECOMMENDATIONS

# PRICING STRATEGY 

# -> Optimize prices for new hosts based on their location, property type, and amenities to remain competitive.
# -> Suggest slightly premium pricing for highly-rated, experienced hosts in high-demand cities.

# FEATURE ENHANCEMENT

# -> Encourage hosts to increase number of amenities — e.g., free Wi-Fi, breakfast, parking — as these are linked to 
# higher prices.
# -> Promote instant booking feature to attract guests and justify higher rates.

# MARKET EXPANSION

# -> Identify cities and property types where demand is high and guide new hosts to invest in those areas.

# HOST DEVELOPEMENT 

# -> Train new hosts on improving their listings, adding amenities, and maintaining high responsiveness to boost both
# bookings and prices.






